In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,3234.23,-0.02,-0.00,0.02,0.00,-0.00,0.00,-0.00,-0.00
3,-0.02,845.36,-0.01,0.00,-0.00,0.00,0.00,0.00,0.00
4,-0.00,-0.01,209.14,0.00,-0.00,-0.00,-0.00,-0.00,0.00
5,0.02,0.00,0.00,49.35,0.00,-0.00,0.00,0.00,0.00
6,0.00,-0.00,-0.00,0.00,11.38,-0.00,-0.00,0.00,0.00
7,-0.00,0.00,-0.00,-0.00,-0.00,2.64,0.00,0.00,0.00
8,0.00,0.00,-0.00,0.00,-0.00,0.00,0.63,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.16,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00003,0.00001,-0.00002,-0.00001
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,0.00007,0.00001,0.00001
4,-0.00000,-0.00001,1.00000,0.00004,-0.00000,-0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00001,-0.00002,0.00007,0.00001,0.00001
6,0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00002,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00001,0.00001
8,0.00001,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00003,0.00001
9,-0.00002,0.00001,-0.00001,0.00001,0.00003,0.00001,0.00003,1.00000,-0.00001
10,-0.00001,0.00001,0.00002,0.00001,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04110815494510115

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.16366308e+06 7.81850051e+04 4.68787856e+03 2.53816015e+02
 1.29687485e+01 6.57480078e-01 3.37169819e-02 1.62591100e-03
 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999354,-0.034990,-0.008028,-0.001881,-0.000434,-0.000102,-0.000024,-0.000006,-0.000002
1,0.034644,0.998645,-0.037873,-0.008338,-0.001921,-0.000444,-0.000105,-0.000026,-0.000007
2,0.009239,0.037173,0.998345,-0.041864,-0.009065,-0.002099,-0.000505,-0.000125,-0.000032
3,0.002524,0.009700,0.040990,0.997848,-0.048957,-0.010691,-0.002541,-0.000633,-0.000163
4,0.000697,0.002672,0.010809,0.047736,0.996793,-0.061626,-0.013811,-0.003416,-0.000883
5,0.000202,0.000769,0.003109,0.013211,0.059561,0.994332,-0.084651,-0.019530,-0.005024
6,0.000063,0.000239,0.000978,0.004124,0.017957,0.080210,0.987499,-0.130828,-0.030903
7,0.000022,0.000084,0.000342,0.001448,0.006280,0.027091,0.116757,0.959241,-0.255831
8,0.000012,0.000046,0.000188,0.000795,0.003449,0.014851,0.062045,0.249695,0.966214


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.000646444219382647,
 0.001354923916085271,
 0.001654695149433505,
 0.0021516485541016594,
 0.003207199206046729,
 0.005668446764184987,
 0.012500621385545307,
 0.04075915237221439,
 0.033785914793482696]